![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### HTTP using Python

# Greetings with HTTP

In this exercise, you will need to construct a function that communicates with a locally running server.  Here we want you only to use only `telnetlib` rather than higher-level client libraries used in later lessons.  In particular, you will connect to port 2551 on `localhost`, and to the resource path `/stream` that may be queried with a GET method.  

As in the example shown in the lesson, data will arrive relatively slowly as lines; but in the server running for this exercise, it will continue infinitely.  You wish to terminate the session from the client side after a greeting longer than 20 characters is encountered. This will take a variable number of lines, since the greetings are randomized. Moreover, your function should itself be a generator function that will basically proxy the server's response.

A correct implementation of the function will behave like this (but with various greeting lines of similar format):

```python
>>> for data in short_greetings():
...     print(data)

(13, 'Goddag Nancy!')
(14, 'Shikamoo Mary!')
(11, 'Salut Mary!')
(11, 'Hallo John!')
(9, 'Yā Nancy!')
(17, 'Namaste Patricia!')
(11, 'Salut Mary!')
(19, 'Dzień dobry Robert!')
(15, 'Selam Patricia!')
(15, 'Goedendag Mary!')
(23, 'Selamat siang Patricia!')
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Initial code

In [8]:
from telnetlib import Telnet

from exercise_server import start
start()

def short_greetings():
    # Telnet(... something ...)
    path = '/stream'
    for _ in range(10):
        greeting = "Hello David!"
        yield (len(greeting), greeting)
        
    # your code goes here
    

---
## Solution

In [9]:
def short_greetings():
    with Telnet('localhost', 2551) as conn:
        conn.interact()
        conn.write(b'GET /stream HTTP/1.1\r\n\r\n')
        while True:
            resp = conn.read_until('⌁'.encode(), 0.1)
            if b'\r\n\r\n' in resp:
                header, resp = resp.split(b'\r\n\r\n')
            elif resp:
                line = resp.decode()
                yield (len(line), line)
                if len(line) > 20:
                    break

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Test Cases

Check your solution by running the following test case:

In [10]:
from itertools import product
import requests

def test_greeting_client():
    seen_long = False
    for length, phrase in short_greetings():
        assert len(phrase) == length, "Length of phrase incorrect"
        if length > 20:
            seen_long = True
        else:
            assert length <= 20 or not seen_long, "Phrase neither short nor final"
    assert seen_long, "No long greeting was encountered!"
    print("Your solution is ok!")
    
test_greeting_client()

Your solution is ok!


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)